# Google Page Rank

In this Jupyter Notebook, we will implement the Google Page Rank algorithm. The algorithm is used to rank web pages based on their importance, considering both the number and quality of incoming links.

## Modeling Internet pages

Since the whole Internet is consisted of the web pages which are linking other web sites and pages, we can represent the whole Internet as a graph defined with matrix.

This matrix, often referred to as the adjacency matrix, captures the relationships between different web pages based on their hyperlinks. Each row and column in the matrix represents a web page, and the entries indicate whether a hyperlink exists between two pages. By representing web pages as a matrix, we can analyze the intricate network of interconnections, enabling us to understand the relationships and navigate the vast web of information effectively. This matrix-based representation serves as a fundamental building block for various algorithms, including the Google Page Rank algorithm, that aim to measure and rank the importance of web pages based on their connectivity patterns.

## Solving the Page Rank

To quantify this for a web of n pages, let $L_k$ ⊂ {1, 2, . . . , n} denote the set of pages with a link to page k, that is, $L_k$ is the set of page k’s backlinks. For each k we require

$$ 
x_k = \sum_{j \in L_k} \frac{x_j}{n_j}
$$

where $n_j$ is the number of outgoing links from page j (which must be positive since if j ∈ $L_k$ , then page j links to at least page k). We will assume that a link from a page to itself will not be counted.

Solving this linear problem can be done using matrix calculation as:

$$
\mathbf{A} \mathbf{x} = \mathbf{x}
$$

One way to look at this problem is as finding the eigenvectors of the matrix $\mathbf{A}$ for the eigenvalue 1 which can be done in mupltiple different ways, but today we are going to use iterative "power" method.

Unfortunatlly, as nice as this seems, there are few problems that can occur in the real world. 
Those problems are:
+ **Subwebs** - Not all parts of the internet are connected to eachother and there will be many dissconnected subwebs that create the whole Internet. However, this results only in dimensionality of our eigenvector; dimension of the eigenvector will be the same as the number of subwebs; each of the dimensions will be the resulting score for each subweb.
+ **Dangling nodes** - Some web pages will have no links to other pages which makes them a dangling node. This causes some problems when solving the Page Rank.

There is a nice way of avoiding this problems while still getting good Page Ranks for the pages. Let's define matrix $\mathbf{S}$ which is n x n with all entries $1/n$. Now we can replace our matrix $\mathbf{A}$ with:

$$
\mathbf{M} = (1 - m)\mathbf{A} + m\mathbf{S}
$$

where \(0 \leq m \leq 1\). This creates matrix with node dangling nodes and now can be used to calculate "importance score" of pages.

> **Note:** This was a simple explanation of how the Page Rang works, but there is a lot more mathematical theory as in why does this work and some definitions of the column-stochastic matrices and other important details that can be read in the "The $25,000,000,000 Eigenvector: The Linear Algebra behind Google" article by Kurt Bryan and Tanya Leise.

## Loading Required Libraries

To begin, we need to import the necessary libraries. In this case, we will be using the `numpy` library for efficient numerical computations.


In [30]:
import numpy as np

## Loading Scraped Matrix

Next, we load the scraped matrix, which represents the connectivity between different web pages. This matrix will serve as the basis for calculating the page ranks.

This matrix was made by scraping "https://solo-leveling.fandom.com" with the maximum depth of 3 so that the matrix isn't too big and has as least as possible dangling nodes (pages that don't link to any other page) and subwebs (smaller parts of web that have no links between each other).

In [31]:
with open('Matrix.txt') as f:
    lines = f.readlines()

    web_sites = np.array(lines[0].strip().split(","))
    weights = []
    for line in lines[1:]:
        weights.append(list(map(lambda x: float(x), line.strip().split(","))))

    M = np.array(weights)
M.shape

(1259, 1259)

## Defining Functions for Calculating Page Scores

We define a set of functions to calculate the scores of the web pages. These functions will help us determine the importance and ranking of each page based on the connectivity information provided by the matrix.
These functions can also be used for determening eigenvalues and eigenvectors for any given matrix.

In [34]:
niter = 1000
n = M.shape[0]
x_0 = np.ones((n, 1))

def get_max_eigenvalue(M, niter=1000):
    # Procedure for finding the largest eigenvalue and corresponding eigenvector
    for _ in range(niter):
        x_k1 = M.dot(x_0)
        x_0 = x_k1 / np.linalg.norm(x_k1)

    alpha = (x_0.T.dot(M).dot(x_0)) / (x_0.T.dot(x_0))
    return x_0, alpha

def get_min_eigenvalue(M, niter=1000):
    # Inverse procedure for finding the smallest eigenvalue and corresponding eigenvector
    n = M.shape[0]
    B = np.linalg.inv(M)
    x_0 = np.ones((n, 1))
    for _ in range(niter):
        x_k1 = B.dot(x_0)
        x_0 = x_k1 / np.linalg.norm(x_k1)

    beta = (x_0.T.dot(B).dot(x_0)) / (x_0.T.dot(x_0))
    smallest_eigenvalue = 1 / beta
    return x_0, smallest_eigenvalue

def get_eigenvalue_omega(M, omega, niter=1000):
    # Procedure for obtaining the eigenvector for a chosen eigenvalue omega
    n = M.shape[0]
    x_0 = np.ones((n, 1))
    for _ in range(niter):
        alpha_prev = (x_0.T.dot(M).dot(x_0)) / (x_0.T.dot(x_0))
        x_k1 = np.linalg.inv(M - omega * np.eye(n)).dot(x_0)
        x_0 = x_k1 / np.linalg.norm(x_k1)
        alpha = (x_0.T.dot(M).dot(x_0)) / (x_0.T.dot(x_0))

        if np.abs(alpha - alpha_prev) < 1e-6:
            break
    eigenvalue_omega = (x_0.T.dot(M).dot(x_0)) / (x_0.T.dot(x_0))
    eigenvector_omega = x_0
    return eigenvector_omega, eigenvalue_omega

def diagonalize(M, niter=1000):
    # Procedure for diagonalizing the matrix using QR decomposition
    A1 = M.copy()
    for _ in range(niter):
        Q, R = np.linalg.qr(A1)
        A1 = R.dot(Q)
    return A1

## Calculating Page Ranks and displaying Top Pages

Using the defined functions, we calculate the page ranks for each web page in the matrix. The page ranks are a measure of the relative importance of each page within the network of pages.

Finally, we display the top pages based on their page ranks. These pages are considered to be the most important and influential within the network.

By implementing the Google Page Rank algorithm, we can gain insights into the importance and ranking of web pages based on their connectivity patterns. Let's proceed with the implementation.

In [38]:
def get_pages(num, page_ranks, pages):
    # Procedure for obtaining the top 10 pages
    n = page_ranks.shape[0]
    pages = pages.reshape((n, 1))
    ranks = np.hstack((pages, page_ranks))
    ranks = ranks[ranks[:, 1].argsort()]
    ranks = ranks[::-1]
    return ranks[:num]

page_ranks, eigen_value = get_eigenvalue_omega(M, 1)
print(f'Eigenvalue: {eigen_value}')
num = 10
best = get_pages(num, page_ranks, web_sites)
print(f'First {num} pages base on Google Page rank system and their scores:\n {best}')

[[0.98737689]]
Eigenvalue: [[0.94263209]]
First 10 pages base on Google Page rank system and their scores:
 [['https://solo-leveling.fandom.com/createnewwiki.fandom.com/Special:CreateNewWiki'
  '0.08233887857265766']
 ['https://solo-leveling.fandom.com/wiki/Local_Sitemap'
  '0.08210631288539828']
 ['https://solo-leveling.fandom.com/f' '0.08199355355402307']
 ['https://solo-leveling.fandom.com/wiki/Iron' '0.08056234850632071']
 ['https://solo-leveling.fandom.com/wiki/Groups' '0.08056234850632071']
 ['https://solo-leveling.fandom.com/wiki/Solo_Leveling_Wiki:Official_Policy'
  '0.08056234850632071']
 ['https://solo-leveling.fandom.com/wiki/Solo_Leveling_Wiki:Improve_Pages'
  '0.08056234850632071']
 ['https://solo-leveling.fandom.com/wiki/Class_Advancement_Quest%27s_Dungeon'
  '0.08056234850632071']
 ['https://solo-leveling.fandom.com/wiki/Races' '0.08056234850632071']
 ['https://solo-leveling.fandom.com/wiki/Double_Dungeon'
  '0.08056234850632071']]
